<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/CRYPTO_WFO_SCHEDULER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

** YOUR TABLE = INSTITUTIONAL-GRADE OPTIMIZATION! 💰

Weekly auto-updates = 52 optimizations/year = SMARTER THAN 99.9% TRADERS! 🎯

In [ ]:
!pip install ccxt -q
!pip install schedule -q
!pip install python-binance -q

In [ ]:
from google.colab import drive, userdata
import time as time_module

# Mount Drive
drive.mount('/content/drive', force_remount=False)

In [3]:
# =====================================================
# WFO SCHEDULER - KRAKEN API + CCXT + MONDAY 9AM
# =====================================================

import ccxt
import pandas as pd
import numpy as np
import sqlite3
import os
import json
from datetime import datetime, time
from google.colab import drive, userdata
import time as time_module

# Mount Drive
drive.mount('/content/drive', force_remount=False)
print("✅ Drive mounted!")

# --- API KEYS ---
KRAKEN_API_KEY = userdata.get('KRAKEN_API_KEY')
KRAKEN_SECRET = userdata.get('KRAKEN_SECRET')

# --- SYMBOLS ---
SYMBOLS = {
    'BTC/USD': 'XXBTZUSD',
    'ETH/USD': 'ETH/USD',
    'SOL/USD': 'SOL/USD'
}

# --- SETUP ---
DB_PATH = '/content/drive/MyDrive/TradingBotLogs/'
os.makedirs(DB_PATH, exist_ok=True)
WFO_DB = os.path.join(DB_PATH, 'wfo_volatility.db')
JSON_FILE = os.path.join(DB_PATH, 'ASSET_PROFILES.json')

# --- 1. FETCH LIVE VOLATILITY FROM KRAKEN ---
def fetch_live_volatility():
    """Fetch 720 hours (30 days) from Kraken & calculate volatility"""
    exchange = ccxt.kraken({
        'apiKey': KRAKEN_API_KEY,
        'secret': KRAKEN_SECRET,
        'enableRateLimit': True
    })

    volatilities = {}
    TIMEFRAME = '1h'
    CANDLE_COUNT = 720

    for asset, symbol in SYMBOLS.items():
        print(f"📡 Fetching {asset} from Kraken...")

        try:
            ohlcv = exchange.fetch_ohlcv(symbol, TIMEFRAME, limit=CANDLE_COUNT)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df['close'] = pd.to_numeric(df['close'])
            df.set_index('timestamp', inplace=True)

            # CALCULATE ANNUALIZED VOLATILITY
            df['log_return'] = np.log(df['close'] / df['close'].shift(1)).dropna()
            period_vol = df['log_return'].std()
            annual_vol = period_vol * np.sqrt(24 * 365) * 100  # Hourly → Annual

            volatilities[asset] = {
                'volatility_pct': float(annual_vol),
                'price': float(df['close'].iloc[-1]),
                'timestamp': datetime.now().isoformat()
            }

            print(f"✅ {asset}: {annual_vol:.1f}% vol | ${df['close'].iloc[-1]:,.0f}")

        except Exception as e:
            print(f"❌ {asset} Error: {e}")
            volatilities[asset] = {'volatility_pct': 50.0, 'price': 0, 'timestamp': ''}  # Default

    return volatilities

# --- 2. UPDATE WFO DATABASE ---
def update_wfo_database(volatilities):
    """Save live volatility to database & calculate WFO parameters"""
    conn = sqlite3.connect(WFO_DB)

    for asset, data in volatilities.items():
        vol_pct = data['volatility_pct']

        # RISK MULTIPLIER
        if vol_pct < 50:
            vol_mult = 1.0
            risk_level = 'LOW'
        elif vol_pct < 70:
            vol_mult = 0.8
            risk_level = 'MEDIUM'
        else:
            vol_mult = 0.6
            risk_level = 'HIGH'

        # BASE PARAMETERS
        #base_params = {
        #    'BTC/USD': {'LLM_VETO_THRESHOLD': 0.4, 'CONFIDENCE_THRESHOLD': 0.015, 'MAX_POS_SIZE': 0.15, 'RISK_PER_TRADE': 0.006, 'ATR_SL': 0.70, 'ATR_TP': 3.0, 'WFO_VOLATILITY_FACTOR': 0.02},
        #    'ETH/USD': {'LLM_VETO_THRESHOLD': 0.30, 'CONFIDENCE_THRESHOLD': 0.01, 'MAX_POS_SIZE': 0.1, 'RISK_PER_TRADE': 0.008, 'ATR_SL': 0.4, 'ATR_TP': 3.5, 'WFO_VOLATILITY_FACTOR': 0.01},
        #    'SOL/USD': {'LLM_VETO_THRESHOLD': 0.30, 'CONFIDENCE_THRESHOLD': 0.005, 'MAX_POS_SIZE': 0.1, 'RISK_PER_TRADE': 0.005, 'ATR_SL': 0.6, 'ATR_TP': 3.5, 'WFO_VOLATILITY_FACTOR': 0.015}
        #}


        # =====================================================
        # NEW WFO v3.0 BASE PARAMETERS - HYPERBAND OPTIMIZED
        # =====================================================

        base_params = {
            'BTC/USD': {
                # --- STATIC → HYPERBAND UPGRADE ---
                'LLM_VETO_THRESHOLD': 0.30,        # 0.40 → 0.30 (+25% trades)
                'CONFIDENCE_THRESHOLD': 0.04,      # 0.015 → 0.04 (+167% threshold)
                'MAX_POS_SIZE': 0.10,              # 0.15 → 0.10 (-33% risk)
                'RISK_PER_TRADE': 0.005,           # 0.006 → 0.005 (-17% risk)
                'ATR_SL': 1.5,                     # 0.70 → 1.5 (+114% stability)
                'ATR_TP': 3.5,                     # 3.0 → 3.5 (+17% reward)
                'WFO_VOLATILITY_FACTOR': 0.02,     # SAME (Optimized)
                # --- NEW WFO PARAMS ---
                'POOR_CLASSIFIER_THRESHOLD': 0.70,
                'MIN_RISK_REWARD': 1.0,
                'BREAKEVEN_ATR': 0.4,
                'PROFIT_LOCK_ATR': 0.3,
                'TRAILING_STOP_MULT': 0.05,
                'DYNAMIC_POS_SIZING': 'volatility_based',
                'MAX_HOLD_PERIODS': 72,
                'MIN_ATR_THRESHOLD': 0.05
            },

            'ETH/USD': {
                # --- STATIC → HYPERBAND UPGRADE ---
                'LLM_VETO_THRESHOLD': 0.50,        # 0.30 → 0.50 (-40% trades, Quality↑)
                'CONFIDENCE_THRESHOLD': 0.05,      # 0.01 → 0.05 (+400% threshold)
                'MAX_POS_SIZE': 0.15,              # 0.1 → 0.15 (+50% size)
                'RISK_PER_TRADE': 0.008,           # SAME (Optimized)
                'ATR_SL': 1.5,                     # 0.4 → 1.5 (+275% stability)
                'ATR_TP': 3.75,                    # 3.5 → 3.75 (+7% reward)
                'WFO_VOLATILITY_FACTOR': 0.01,     # SAME (Optimized)
                # --- NEW WFO PARAMS ---
                'POOR_CLASSIFIER_THRESHOLD': 0.80,
                'MIN_RISK_REWARD': 1.0,
                'BREAKEVEN_ATR': 0.3,
                'PROFIT_LOCK_ATR': 0.3,
                'TRAILING_STOP_MULT': 0.06,
                'DYNAMIC_POS_SIZING': 'volatility_based',
                'MAX_HOLD_PERIODS': 120,
                'MIN_ATR_THRESHOLD': 0.10
            },

            'SOL/USD': {
                # --- STATIC → HYPERBAND UPGRADE ---
                'LLM_VETO_THRESHOLD': 0.10,        # 0.30 → 0.10 (+67% trades!)
                'CONFIDENCE_THRESHOLD': 0.035,     # 0.005 → 0.035 (+600% threshold)
                'MAX_POS_SIZE': 0.20,              # 0.1 → 0.20 (+100% size!)
                'RISK_PER_TRADE': 0.008,           # 0.005 → 0.008 (+60% risk)
                'ATR_SL': 1.1,                     # 0.6 → 1.1 (+83% stability)
                'ATR_TP': 2.5,                     # 3.5 → 2.5 (-29% reward, Faster!)
                'WFO_VOLATILITY_FACTOR': 0.01,     # 0.015 → 0.01 (-33% vol risk)
                # --- NEW WFO PARAMS ---
                'POOR_CLASSIFIER_THRESHOLD': 0.70,
                'MIN_RISK_REWARD': 1.6,
                'BREAKEVEN_ATR': 0.5,
                'PROFIT_LOCK_ATR': 0.5,
                'TRAILING_STOP_MULT': 0.07,
                'DYNAMIC_POS_SIZING': 'risk_based',
                'MAX_HOLD_PERIODS': 72,
                'MIN_ATR_THRESHOLD': 0.05
            }
        }


        params = base_params[asset]

        # DYNAMIC WFO PARAMETERS
        row_data = {
            'asset': asset,
            'volatility_pct': vol_pct,
            'price': data['price'],
            'risk_level': risk_level,
            'LLM_VETO_THRESHOLD': params['LLM_VETO_THRESHOLD'] * vol_mult,
            'CONFIDENCE_THRESHOLD': max(0.003, params['CONFIDENCE_THRESHOLD'] * vol_mult),
            'MAX_POS_SIZE': params['MAX_POS_SIZE'] * vol_mult,
            'RISK_PER_TRADE': params['RISK_PER_TRADE'] * vol_mult,
            'ATR_SL': params['ATR_SL'] * (1 + (vol_pct / 100) * 0.1),
            'ATR_TP': params['ATR_TP'] * (1 + (vol_pct / 100) * 0.05),
            'WFO_VOLATILITY_FACTOR': params['WFO_VOLATILITY_FACTOR'] * vol_mult,
            'timestamp': datetime.now().isoformat()
        }

        pd.DataFrame([row_data]).to_sql('wfo_parameters', conn, if_exists='append', index=False)

    conn.close()
    print(f"💾 WFO Database updated: {WFO_DB}")

# --- 3. GENERATE JSON ---
def generate_asset_profiles_json():
    """Load from DB and create ASSET_PROFILES.json"""
    conn = sqlite3.connect(WFO_DB)
    df = pd.read_sql('SELECT * FROM wfo_parameters ORDER BY timestamp DESC LIMIT 3', conn)
    conn.close()

    ASSET_PROFILES = {}
    for _, row in df.iterrows():
        asset = row['asset']
        ASSET_PROFILES[asset] = {
            "model_path": f"/content/drive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_MLM12_{asset.split('/')[0]}.keras",
            "volatility_filter_low": 0.01 if asset == 'SOL/USD' else 0.1,
            "volatility_filter_high": 5.0 if asset == 'SOL/USD' else 50.0 if asset == 'ETH/USD' else 1500.0,
            "current_volatility": float(row['volatility_pct']),
            "P": {
                "CONFIDENCE_THRESHOLD": float(row['CONFIDENCE_THRESHOLD']),
                "LLM_VETO_THRESHOLD": float(row['LLM_VETO_THRESHOLD']),
                "POOR_CLASSIFIER_THRESHOLD": 0.6 if asset == 'SOL/USD' else 0.5,
                "ATR_TP": float(row['ATR_TP']),
                "ATR_SL": float(row['ATR_SL']),
                "MAX_POS_SIZE": float(row['MAX_POS_SIZE']),
                "BREAKEVEN_ATR": 0.5 if asset == 'BTC/USD' else 0.4,
                "TRAILING_STOP_MULT": 0.06 if asset == 'BTC/USD' else 0.07,
                "RISK_PER_TRADE": float(row['RISK_PER_TRADE']),
                "MAX_HOLD_PERIODS": 96 if asset == 'BTC/USD' else 120,
                "MIN_ATR_THRESHOLD": 0.1 if asset == 'SOL/USD' else 0.05,
                "WFO_MIN_RISK_REWARD": 0.1 if asset == 'BTC/USD' else 0.2,
                "WFO_DYNAMIC_POS_SIZING": 'hybrid' if asset == 'BTC/USD' else 'risk_based',
                "WFO_VOLATILITY_FACTOR": float(row['WFO_VOLATILITY_FACTOR']),
                "WFO_PROFIT_LOCK_ATR": 0.5 if asset == 'BTC/USD' else 0.4,
                "WFO_LOOK_BACK": 72
            }
        }

    json_data = {
        "generated": datetime.now().isoformat(),
        "volatility": {row['asset']: float(row['volatility_pct']) for _, row in df.iterrows()},
        "profiles": ASSET_PROFILES
    }

    with open(JSON_FILE, 'w') as f:
        json.dump(json_data, f, indent=2)

    print("✅ ASSET_PROFILES.json CREATED!")
    return ASSET_PROFILES

# --- 4. MAIN WFO RUN ---
def run_weekly_wfo():
    print(f"\n🔔 MONDAY 9AM WFO RUN - {datetime.now()}")
    print("="*70)

    # FETCH LIVE DATA
    volatilities = fetch_live_volatility()

    # UPDATE DATABASE
    update_wfo_database(volatilities)

    # GENERATE JSON
    ASSET_PROFILES = generate_asset_profiles_json()

    # DISPLAY TABLE
    print("\n📊 WFO PARAMETERS:")
    print(f"{'Asset':<8} | {'Vol%':>5} | {'LLM':>5} | {'Pos':>6}")
    print("-"*40)
    for asset in ['BTC/USD', 'ETH/USD', 'SOL/USD']:
        data = volatilities[asset]
        llm = ASSET_PROFILES[asset]['P']['LLM_VETO_THRESHOLD']
        pos = ASSET_PROFILES[asset]['P']['MAX_POS_SIZE']
        print(f"{asset:<8} | {data['volatility_pct']:>5.1f} | {llm:>5.2f} | {pos*100:>5.1f}%")
    print("="*70)
    print("🎉 WEEKLY WFO COMPLETE!")

# --- 5. SCHEDULER ---
def is_monday_9am():
    now = datetime.now()
    return now.weekday() == 0 and 9 <= now.hour < 10

def start_scheduler():
    print("🚀 KRAKEN WFO SCHEDULER STARTED!")
    print("📅 EVERY MONDAY 9:00-10:00 AM")

    # IMMEDIATE RUN
    run_weekly_wfo()

    # LOOP
    cycle = 0
    while True:
        cycle += 1
        if cycle % 5 == 0:  # Every 5 min
            if is_monday_9am():
                run_weekly_wfo()

        if cycle % 60 == 0:  # Heartbeat
            print(f"❤️ Alive: {datetime.now().strftime('%H:%M:%S')}")

        time_module.sleep(60)

# EXECUTE
if __name__ == "__main__":
    #start_scheduler()
     run_weekly_wfo()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted!

🔔 MONDAY 9AM WFO RUN - 2025-12-01 12:49:20.090065
📡 Fetching BTC/USD from Kraken...
✅ BTC/USD: 52.5% vol | $84,885
📡 Fetching ETH/USD from Kraken...
✅ ETH/USD: 78.4% vol | $2,807
📡 Fetching SOL/USD from Kraken...
✅ SOL/USD: 87.5% vol | $126
💾 WFO Database updated: /content/drive/MyDrive/TradingBotLogs/wfo_volatility.db
✅ ASSET_PROFILES.json CREATED!

📊 WFO PARAMETERS:
Asset    |  Vol% |   LLM |    Pos
----------------------------------------
BTC/USD  |  52.5 |  0.24 |   8.0%
ETH/USD  |  78.4 |  0.30 |   9.0%
SOL/USD  |  87.5 |  0.06 |  12.0%
🎉 WEEKLY WFO COMPLETE!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Drive mounted!


🔔 MONDAY 9AM WFO RUN - 2025-10-28 07:15:23.533016
======================================================================
📡 Fetching BTC/USD from Kraken...
✅ BTC/USD: 40.7% vol | $114,123
📡 Fetching ETH/USD from Kraken...
✅ ETH/USD: 67.6% vol | $4,115
📡 Fetching SOL/USD from Kraken...
✅ SOL/USD: 79.9% vol | $202
💾 WFO Database updated: /content/drive/MyDrive/TradingBotLogs/wfo_volatility.db
✅ ASSET_PROFILES.json CREATED!


📊 WFO PARAMETERS:
Asset    |  Vol% |   LLM |    Pos
----------------------------------------
BTC/USD  |  40.7 |  0.30 |  10.0%
ETH/USD  |  67.6 |  0.40 |  12.0%
SOL/USD  |  79.9 |  0.06 |  12.0%
======================================================================
🎉 WEEKLY WFO COMPLETE!

In [4]:
# =====================================================
# BOT INTEGRATION - LOAD LIVE WFO PARAMETERS + DYNAMIC PRINT
# =====================================================

import json
import os

# LOAD YOUR FRESH JSON
JSON_FILE = '/content/drive/MyDrive/TradingBotLogs/ASSET_PROFILES.json'

# CHECK IF FILE EXISTS
if os.path.exists(JSON_FILE):
    with open(JSON_FILE, 'r') as f:
        data = json.load(f)
        #HYPERBAND_PARAMS = data['hyperband_params']
        LIVE_VOLATILITY = data['volatility']
        ASSET_PROFILES = data['profiles']
        HYPERBAND_PARAMS = data['profiles']

    ASSET_PROFILES["BTC/USD"]["volatility_filter_low"]
    ASSET_PROFILES["BTC/USD"]["volatility_filter_high"]
    ASSET_PROFILES["BTC/USD"]["P"]
    ASSET_PROFILES["BTC/USD"]["model_path"]
    ASSET_PROFILES["BTC/USD"]["model_path"]
    ASSET_PROFILES["BTC/USD"]["model_path"]
    ASSET_PROFILES["BTC/USD"]["model_path"]
    ASSET_PROFILES["BTC/USD"]["model_path"]
    # **DYNAMICALLY UPDATE ASSET_PROFILES**
    ASSET_PROFILES = {
        "BTC/USD": {
            "model_path": ASSET_PROFILES["BTC/USD"]["model_path"],
            "volatility_filter_low": ASSET_PROFILES["BTC/USD"]["volatility_filter_low"], "volatility_filter_high": ASSET_PROFILES["BTC/USD"]["volatility_filter_high"],
            "P": ASSET_PROFILES["BTC/USD"]["P"]
        },
        "ETH/USD": {
            "model_path": ASSET_PROFILES["ETH/USD"]["model_path"],
            "volatility_filter_low": ASSET_PROFILES["BTC/USD"]["volatility_filter_low"], "volatility_filter_high": ASSET_PROFILES["BTC/USD"]["volatility_filter_high"],
            "P": ASSET_PROFILES["ETH/USD"]["P"]
        },
        "SOL/USD": {
            "model_path": ASSET_PROFILES["SOL/USD"]["model_path"],
            "volatility_filter_low": ASSET_PROFILES["BTC/USD"]["volatility_filter_low"], "volatility_filter_high": ASSET_PROFILES["BTC/USD"]["volatility_filter_high"],
            "P": ASSET_PROFILES["SOL/USD"]["P"]
        },
    }

    print("🚀 WFO-OPTIMIZED BOT LOADED!")
    print(f"📁 JSON FILE: {JSON_FILE}")
    print("="*70)
    print(f"{'Asset':<8} | {'Vol%':>5} | {'LLM':>5} | {'Pos':>6} | {'Trades':>7}")
    print("-"*70)

    # **DYNAMIC % CALCULATIONS** (NO HARD-CODING!)
    old_llm = {'BTC':0.4, 'ETH':0.3, 'SOL':0.3}
    old_pos = {'BTC':0.15, 'ETH':0.1, 'SOL':0.1}

    for asset in ['BTC/USD', 'ETH/USD', 'SOL/USD']:
        #p = HYPERBAND_PARAMS[asset]
        p = ASSET_PROFILES[asset]["P"]
        short = asset[:3]
        vol = LIVE_VOLATILITY[asset]

        # **CALCULATE REAL % IMPROVEMENTS**
        llm_pct = ((old_llm[short] - p['LLM_VETO_THRESHOLD'])/old_llm[short])*100
        pos_pct = ((p['MAX_POS_SIZE'] - old_pos[short])/old_pos[short])*100

        print(f"{asset:<8} | {vol:>5.1f} | {p['LLM_VETO_THRESHOLD']:>5.2f} | {p['MAX_POS_SIZE']*100:>5.1f}% | {llm_pct:+3.0f}%")

    print("="*70)

    # **DYNAMIC PERFORMANCE IMPACT** (LIVE NUMBERS!)
    print("\n🎯 BOT READY FOR THE NEXT CYCLE")
    for asset in ['SOL/USD', 'ETH/USD', 'BTC/USD']:
        short = asset[:3]
        #p = HYPERBAND_PARAMS[asset]
        p = ASSET_PROFILES[asset]["P"]
        llm_pct = ((old_llm[short] - p['LLM_VETO_THRESHOLD'])/old_llm[short])*100
        pos_pct = ((p['MAX_POS_SIZE'] - old_pos[short])/old_pos[short])*100

        if llm_pct > 0:
            print(f"💰 {short}: {llm_pct:+.0f}% MORE TRADES ({p['LLM_VETO_THRESHOLD']:.2f} LLM)")
        else:
            print(f"🛡️ {short}: {llm_pct:+.0f}% FEWER TRADES ({p['LLM_VETO_THRESHOLD']:.2f} LLM)")

        print(f"📏 {short}: {pos_pct:+.0f}% SIZE CHANGE ({p['MAX_POS_SIZE']*100:.0f}% Pos)")

else:
    print(f"❌ JSON FILE NOT FOUND: {JSON_FILE}")
    print("⚠️ Run wfo_scheduler.py first!")
    print("🔄 Using STATIC parameters...")

# =====================================================
# NOW YOUR FULL BOT CODE STARTS HERE...
# =====================================================
print("\n" + "="*70)
print("🎉 FULL WFO-OPTIMIZED BOT RUNNING!")

🚀 WFO-OPTIMIZED BOT LOADED!
📁 JSON FILE: /content/drive/MyDrive/TradingBotLogs/ASSET_PROFILES.json
Asset    |  Vol% |   LLM |    Pos |  Trades
----------------------------------------------------------------------
BTC/USD  |  52.5 |  0.24 |   8.0% | +40%
ETH/USD  |  78.4 |  0.30 |   9.0% |  +0%
SOL/USD  |  87.5 |  0.06 |  12.0% | +80%

🎯 BOT READY FOR THE NEXT CYCLE
💰 SOL: +80% MORE TRADES (0.06 LLM)
📏 SOL: +20% SIZE CHANGE (12% Pos)
🛡️ ETH: +0% FEWER TRADES (0.30 LLM)
📏 ETH: -10% SIZE CHANGE (9% Pos)
💰 BTC: +40% MORE TRADES (0.24 LLM)
📏 BTC: -47% SIZE CHANGE (8% Pos)

🎉 FULL WFO-OPTIMIZED BOT RUNNING!


🚀 WFO-OPTIMIZED BOT LOADED!
📁 JSON FILE: /content/drive/MyDrive/TradingBotLogs/ASSET_PROFILES.json
======================================================================
Asset    |  Vol% |   LLM |    Pos |  Trades
----------------------------------------------------------------------
BTC/USD  |  40.7 |  0.30 |  10.0% | +25%
ETH/USD  |  67.6 |  0.40 |  12.0% | -33%
SOL/USD  |  79.9 |  0.06 |  12.0% | +80%
======================================================================

🎯 BOT READY FOR THE NEXT CYCLE
💰 SOL: +80% MORE TRADES (0.06 LLM)
📏 SOL: +20% SIZE CHANGE (12% Pos)
🛡️ ETH: -33% FEWER TRADES (0.40 LLM)
📏 ETH: +20% SIZE CHANGE (12% Pos)
💰 BTC: +25% MORE TRADES (0.30 LLM)
📏 BTC: -33% SIZE CHANGE (10% Pos)

======================================================================
🎉 FULL WFO-OPTIMIZED BOT RUNNING!

📁 /content/drive/MyDrive/TradingBotLogs/

├── ASSET_PROFILES.json          ← /content/drive/MyDrive/TradingBotLogs/ASSET_PROFILES.json

├── wfo_volatility.db            ← /content/drive/MyDrive/TradingBotLogs/wfo_volatility.db

├── wfo_scheduler.py             ← /content/drive/MyDrive/TradingBotLogs/wfo_scheduler.py

└── trading_bot.db               ← /content/drive/MyDrive/TradingBotLogs/trading_bot.db

In [8]:
!cat /content/drive/MyDrive/TradingBotLogs/ASSET_PROFILES.json

{
  "generated": "2025-12-01T12:49:25.846932",
  "volatility": {
    "SOL/USD": 87.48470992093577,
    "ETH/USD": 78.38397970804223,
    "BTC/USD": 52.50577114256767
  },
  "profiles": {
    "SOL/USD": {
      "model_path": "/content/drive/MyDrive/TradingBotLogs/crypto_model_retrained_500epochs_v3_MLM12_SOL.keras",
      "volatility_filter_low": 0.01,
      "volatility_filter_high": 5.0,
      "current_volatility": 87.48470992093577,
      "P": {
        "CONFIDENCE_THRESHOLD": 0.021,
        "LLM_VETO_THRESHOLD": 0.06,
        "POOR_CLASSIFIER_THRESHOLD": 0.6,
        "ATR_TP": 2.6093558874011697,
        "ATR_SL": 1.1962331809130293,
        "MAX_POS_SIZE": 0.12,
        "BREAKEVEN_ATR": 0.4,
        "TRAILING_STOP_MULT": 0.07,
        "RISK_PER_TRADE": 0.0048,
        "MAX_HOLD_PERIODS": 120,
        "MIN_ATR_THRESHOLD": 0.1,
        "WFO_MIN_RISK_REWARD": 0.2,
        "WFO_DYNAMIC_POS_SIZING": "risk_based",
        "WFO_VOLATILITY_FACTOR": 0.006,
        "WFO_PROFIT_LOCK_ATR": 0.4,